In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import numpy as np
import pandas as pd
import math
# Used to save logs on file
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras import Sequential
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Used for L2 regularization
from tensorflow.keras import regularizers
# Used for dropout
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import SGD, Adamax, RMSprop, Adagrad, Adam
from sklearn.model_selection import GridSearchCV
from keras.constraints import maxnorm
from tensorflow.keras.losses import MeanSquaredError

from keras import backend as K

import tensorflow as tf


from datetime import datetime

# Used for early stopping
from tensorflow.keras.callbacks import EarlyStopping

from sklearn import preprocessing

# Logging
log = open("NN-grid-CUP-2-P.txt", "w")

# Read and view TR
# dataset = "cup/ML-CUP20-TR.csv"
dataset="/content/drive/My Drive/Machine Learning Project/CUP/data/ML-CUP20-TR.csv"

# Column 0:     ID
# Column 1-10:  inputs
# Column 11-12: target_x target_y
train_names = ["id", "input_1", "input_2", "input_3", "input_4", "input_5", "input_6", "input_7", "input_8", "input_9",
               "input_10", "target_x", "target_y"]
df_train = pd.DataFrame(pd.read_csv(dataset, sep=',', skiprows=7, header=None, names=train_names, index_col=0))

# Store the number of training patterns
n_TR_patterns = len(df_train.index)
print(f'N. rows TR: {n_TR_patterns}\n')

# log.write(f'N. rows TR: {n_TR_patterns}\n')

# print(df_train.head())

# Read and view TS
# dataset = "cup/ML-CUP20-TS.csv"
dataset="/content/drive/My Drive/Machine Learning Project/CUP/data/ML-CUP20-TS.csv"

# Column 0:    ID
# Column 1-10: inputs
test_names = ["id", "input_1", "input_2", "input_3", "input_4", "input_5", "input_6", "input_7", "input_8", "input_9",
              "input_10"]
df_test = pd.DataFrame(pd.read_csv(dataset, sep=',', skiprows=7, header=None, names=test_names, index_col=0))

# Store the number of (blind) testing patterns
n_TS_patterns = len(df_test.index)
print(f'N. rows TS: {n_TS_patterns}\n')

# Divide into TR, VL, and (internal) TS
attributes = [col for col in df_train.columns if 'input' in col]

# Separating internal TS from the rest of training dataset (Hold out ca. 20%)
print("\ndf_train ", len(df_train))  # 1524

df_subset = df_train.sample(200, random_state=100)
print("\ndf_subset ", len(df_subset))  # 304
print(df_subset.head())

df_train = df_train.drop(df_subset.index)
print("\ndf_train after removal ", len(df_train))  # 1220

X_train = df_train[attributes].values
y_train = df_train[['target_x', 'target_y']].values


n_features = X_train.shape[1]
print(X_train.shape[0])

print(f'\nN. input features: {n_features}')

# log.write(f'\nN. input features: {n_features}')

# Handle (blind) TS
attributes = [col for col in df_test.columns if 'input' in col]
X_blind_TS = df_test[attributes].values


X_test = df_subset[attributes].values
y_test = df_subset[['target_x', 'target_y']].values

def mee_keras(y_true, y_pred):
    return K.mean(K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True)))

from sklearn.metrics import make_scorer
my_func = make_scorer(mee_keras, greater_is_better=False)


def r2_keras(y_true, y_pred):
    ss_res = K.sum(K.square(y_true-y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - ss_res/(ss_tot + K.epsilon())


def build_model(neurons=1, dropout_rate=0.0, weight_constraint=0, activation='sigmoid', learn_rate=0.001, momentum=0,
                reg_value=0, optimizer='adam', init_mode='uniform'):
    tf.keras.backend.clear_session()
    # Build the model
    model_1 = Sequential()

    if reg_value != 0:
        # Apply regularization
        layer_1 = Dense(neurons, input_dim=n_features, activation=activation,
                        kernel_initializer=init_mode,
                        kernel_constraint=maxnorm(weight_constraint),
                        kernel_regularizer=regularizers.l2(reg_value),
                        bias_regularizer=regularizers.l2(reg_value), )
        layer_2 = Dense(neurons, activation=activation,
                        kernel_initializer=init_mode,
                        kernel_constraint=maxnorm(weight_constraint),
                        kernel_regularizer=regularizers.l2(reg_value),
                        bias_regularizer=regularizers.l2(reg_value), )
        layer_3 = Dense(2, activation='linear')
    else:
        # No regularization
        layer_1 = Dense(neurons, input_dim=n_features, activation=activation)
        layer_2 = Dense(neurons, activation=activation)
        layer_3 = Dense(2, activation='linear')

    model_1.add(layer_1)

    if dropout_rate != 0:
        # Add dropout, seed for reproducibility
        model_1.add(Dropout(dropout_rate, seed=123))

    model_1.add(layer_2)

    if dropout_rate != 0:
        # Add dropout, seed for reproducibility
        model_1.add(Dropout(dropout_rate, seed=123))

    model_1.add(layer_3)

    # Compile the model
    if optimizer == 'SGD':
        # opt = SGD(learning_rate=learn_rate, momentum=momentum)
        # With clipping
        opt = SGD(learning_rate=learn_rate, momentum=momentum, clipnorm=1.0)
    elif optimizer == 'Adamax':
        # opt = Adamax(learning_rate=learn_rate, name="Adamax")
        # With clipping
        opt = Adamax(learning_rate=learn_rate, name="Adamax", clipnorm=1.0)
    elif optimizer == 'RMSprop':
        # With clipping
        opt = RMSprop(learning_rate=learn_rate, rho=0.9, momentum=momentum,
                      epsilon=1e-07, centered=False, name="RMSprop", clipnorm=1.0)
    elif optimizer == 'Adagrad':
        # With clipping
        opt = Adagrad(learning_rate=learn_rate, initial_accumulator_value=0.1,
                      epsilon=1e-07, name="Adagrad", clipnorm=1.0)
    else:
        # Adam
        # With clipping
        opt = Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999,
                   epsilon=1e-07, amsgrad=False, name="Adam", clipnorm=1.0)

    # Metrics: MSE, MEE (manual implementation), R^2 (manual implementation)
    model_1.compile(loss=MeanSquaredError(), optimizer=opt,
                    metrics=[tf.keras.metrics.MeanSquaredError(), mee_keras, r2_keras])

    return model_1


# HYPER-PARAMETER TUNING
model = KerasRegressor(build_fn=build_model, verbose=0)

learn_rate = [0.1]
momentum = [0.1]
optimizer = ['SGD']
init_mode = ['he_uniform']
batch_size = [50]
epochs = [500]
weight_constraint = [1]
dropout_rate = [0.0]
neurons = [300]
activation = ['tanh']
reg_value = [0.01]


# datetime object containing current date and time
print('\ninizio:', datetime.now())

param_grid = dict(neurons=neurons, dropout_rate=dropout_rate, weight_constraint=weight_constraint, optimizer=optimizer,
                  batch_size=batch_size, epochs=epochs, learn_rate=learn_rate, momentum=momentum, init_mode=init_mode,
                  activation=activation, reg_value = reg_value)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=5, cv=3, scoring='neg_mean_squared_error')

grid_result = grid.fit(X_train, y_train)

print('\nfine:', datetime.now())

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

N. rows TR: 1524

N. rows TS: 472


df_train  1524

df_subset  200
       input_1   input_2   input_3  ...  input_10   target_x   target_y
id                                  ...                                
801   1.155300  0.059795 -0.604900  ...  1.727382  39.929663 -29.595517
720  -0.918605  0.637450  1.347002  ... -0.903759  72.384610 -28.855841
1059 -0.652593  1.394323  0.103282  ... -1.030427  52.796461 -39.445597
189   1.439956 -0.605409 -0.758339  ...  1.819379  40.553341 -30.551312
229   0.915644 -1.250489 -1.028318  ...  0.055362  31.467524 -13.706777

[5 rows x 12 columns]

df_train after removal  1324
1324

N. input features: 10

inizio: 2021-08-20 09:39:59.030991

fine: 2021-08-20 09:42:45.522999
Best: -9.035889 using {'activation': 'tanh', 'batch_size': 50, 'dropout_rate': 0.0, 'epochs': 500, 'init_mode': 'he_uniform', 'learn_rate': 0.1, 'momentum': 0.1, 'neurons': 300, 'optimizer': 'SGD', 'reg_value': 0.01, 'weight_constraint': 1}
-9.035889 (0.719459) with: {'activati

In [37]:
best_grid = grid.best_estimator_

ypred = best_grid.predict(X_test)

mse = mean_squared_error(y_test, ypred)
mee = mee_keras(y_test, ypred)
print("MSE on test set: ", mse)
print("MEE on test set: ", float(mee))

MSE on test set:  7.0344479597085146
MEE on test set:  3.0564269207791015
